In [ ]:
# Imports
import logging
import pandas as pd
import requests 
import sqlite3
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error	
import math	

#logging info
logging.basicConfig(level = logging.DEBUG)



dbName = "../rest_server/medisch_centrum_randstad/db.sqlite3"
tableName = "rest_api_netlify"
url = "http://localhost:8080/medish_centrum_randstad/api/netlify?page=1"
csvFile = "rest_server_new/medisch_centrum_randstad/data/data.csv"

##########################
### Read from REST API ###
##########################
# logging.info("Extract from REST API")
# response = requests.get(url)
# file_contents= response.json()  #dictionary
# logging.info("Load API Response into Pandas dataframe")
# df = pd.DataFrame.from_dict(file_contents['data']) #all the needed info was condensed into one data column called 'data'
# df3 = df.copy() #keep original for df3 
# logging.debug(df.head())
# logging.info("Load dataset in Database")

######################
### Read from .csv ###
######################
#df pd.read_csv('csvFile',skipinitialspace=True)
#df3 = df.copy() #keep original for df3 
#logging.info("Load dataset in Database")

#


In [ ]:
###########################
### Read from SQlite3 db ###
############################
dbConnection = sqlite3.connect(dbName)


In [ ]:
# # df1

# #query db and write to pd:
# dfFromDB = pd.read_sql_query(f"SELECT * FROM {'df1'}", dbConnection)
# #sql adds index, remove:
# df1 = dfFromDB.drop('index', axis=1)
# pd.set_option('display.max_columns', 10)
# display(df1.head())
# # display(df1.info)# 

In [ ]:
#query db and write to pd:
dfFromDB = pd.read_sql_query(f"SELECT * FROM {'df2'}", dbConnection)
#sql adds index, remove:
df2 = dfFromDB.drop('index', axis=1)
pd.set_option('display.max_columns', 10)
display(df2.head(2))
display(df2.info)


,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2,35.6


<bound method DataFrame.info of       genetic  length   mass  exercise  smoking  alcohol  lifespan  sugar  \
0        73.9   185.0   99.7       0.9      0.0      2.4      73.1    6.9   
1        86.0   172.0  105.4       1.8      8.1      0.4      85.0    4.2   
2        83.3   176.0  111.4       1.1      0.8      4.6      81.6    7.5   
3        82.8   164.0  111.4       4.7     11.8      1.0      81.0    2.9   
4        78.7   178.0   71.6       1.5      8.3      4.9      75.0    5.5   
...       ...     ...    ...       ...      ...      ...       ...    ...   
4079     80.3   176.0  115.9       3.7      0.1      5.6      80.7    5.8   
4080     75.3   162.0  129.6       3.6      6.2      5.6      71.1    7.6   
4081     93.1   189.0  115.3       2.7      9.4      4.9      90.0    6.6   
4082     77.9   170.0   90.8       2.3     13.2      0.8      75.9    6.2   
4083    101.3   188.0  129.2       1.2      8.2      6.0      96.9    6.2   

       BMI  
0     29.1  
1     35.6  
2   

In [ ]:
display(df2.describe())

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
count,4084.000000,4084.000000,4084.000000,4084.000000,4084.000000,4084.000000,4084.000000,4084.000000,4084.000000
mean,82.278624,183.885162,92.275930,2.403477,9.809035,2.266846,80.177693,6.487169,27.682076
std,7.441333,12.622858,25.801031,1.170426,6.918614,1.801044,7.840508,2.708622,8.711789
min,63.900000,154.000000,50.000000,0.100000,0.000000,0.000000,57.000000,0.700000,11.300000
25%,77.600000,175.000000,71.475000,1.500000,3.375000,0.600000,75.000000,4.400000,20.900000
50%,81.800000,184.000000,89.500000,2.200000,9.500000,1.700000,79.700000,6.300000,26.500000
75%,86.700000,193.000000,110.825000,3.200000,15.900000,4.000000,85.200000,8.400000,33.200000
max,102.200000,214.000000,163.600000,5.500000,22.200000,6.000000,105.100000,13.800000,65.900000


In [ ]:
# # df3

# #query db and write to pd:
# dfFromDB = pd.read_sql_query(f"SELECT * FROM {'df3'}", dbConnection)
# #sql adds index, remove:
# df3 = dfFromDB.drop('index', axis=1)
# pd.set_option('display.max_columns', 10)
# # display(df3.head())
# # display(df3.info)
# # display(df3.describe())

In [ ]:
dbConnection.close()

In [ ]:
# the IQR clipping for outliers 
df=df2

# Computing IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Clipping the IQR*|15.*IQD|
df2 = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

# make spit for df1

In [ ]:
# functie om coefficient of determination(R²) vanilla: te berekenen
# StDek

def coefficient_R(x):
        # R^2
    train, test = train_test_split(x, test_size=0.2, random_state=42) # random state 42 om steeds de zelfde split te maken.
    
    X = train.drop(['lifespan'], axis=1) # definieer de Xas. Drop de waarde die op de Y-as komt
    y = train.lifespan # de waarde die op de y-as komt. 
    regr = linear_model.LinearRegression()
    regr.fit(X, y) 
    score = regr.score(test.drop(['lifespan'],axis=1),test.lifespan)
    print(f'coefficient of determination(R\N{SUPERSCRIPT TWO}) vanilla:', score) # print de waarde die we willen zien. 
    
    # RMSE 	

    p_test = regr.predict(x.drop(columns='lifespan'))	
	
    mse = mean_squared_error(x.lifespan, p_test)	
    
    rmse = (math.sqrt(mse))	
    print(f'Interpret Root Mean Square Error: {rmse}')	

In [ ]:
# test veld

# df = df2.copy()
# coefficient_R(df)

In [ ]:

# x_train, x_test, y_train, y_test = train_test_s[df1.drop(['lifespan'], axis=1)]plit(df1, test_size=0.2, random_state=42)
# X = x_train 
# y = y_train.lifespan
# regr = linear_model.LinearRegression()

# regr.fit(X,y)

In [ ]:
# df1

# create the df to be trained.
df = df2.drop(['BMI'], axis=1).copy()
 
coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9799785196963832
Interpret Root Mean Square Error: 1.118016753865739


In [ ]:
# df2

# create the df to be trained.
df = df2.copy()
 
coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9812233992575078
Interpret Root Mean Square Error: 1.0758166035183905


In [ ]:
# df3

# create the df to be trained.
df = df2.copy()

## the IQR clipping for outliers 
# Computing IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Clipping the IQR*|15.*IQD|
mean = df.mean()
df = df.clip(lower=mean - 1.5 * IQR, upper=mean + 1.5 * IQR, axis=1)
 

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9735737147751526
Interpret Root Mean Square Error: 1.1994825244839864


<!-- train, test = train_test_split(temp, test_size=0.2, random_state=42)
 
X = train[['genetic', 'length', 'mass', 'exercise', 'smoking', 'alcohol', 'sugar']]
y = train.lifespan
regr = linear_model.LinearRegression()
regr.fit(X, y) 
score = regr.score(test[['genetic', 'length', 'mass', 'exercise', 'smoking', 'alcohol', 'sugar']],test.lifespan)
print(f'coefficient of determination(R\N{SUPERSCRIPT TWO}) vanilla:', score)
a1=score -->

In [ ]:
# df4 drop outliers

# create the df to be trained.
df = df2.copy()

## the IQR clipping for outliers 
# Computing IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Clipping the IQR*|15.*IQD|
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]


coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9820618333051058
Interpret Root Mean Square Error: 1.0543180939446808


In [ ]:
# DF2 met X = df.drop(['index', 'mass', 'length', 'lifespan'], axis=1)

# create the df to be trained.
df = df2.drop(['mass','length'], axis=1).copy()
 
coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9806993146035271
Interpret Root Mean Square Error: 1.0977088924974763


In [ ]:
# df5

# create the df to be trained.
df = df2[['lifespan','genetic','BMI']].copy()
 
coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9077339058831414
Interpret Root Mean Square Error: 2.363476437120093


In [ ]:
# DF2 genetic

# create the df to be trained.
df = df2[['lifespan','genetic']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.900186076846281
Interpret Root Mean Square Error: 2.4527784807396076


In [ ]:
# DF2 length

# create the df to be trained.
df = df2[['lifespan','length']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: -0.0017754178821400846
Interpret Root Mean Square Error: 7.8394018921993744


In [ ]:
# DF2 mass
# create the df to be trained.
df = df2[['lifespan','mass']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.002120888004842225
Interpret Root Mean Square Error: 7.822817425620948


In [ ]:
# DF2 exercise
# create the df to be trained.
df = df2[['lifespan','exercise']].copy()

coefficient_R(df)


coefficient of determination(R²) vanilla: 0.009788416932027255
Interpret Root Mean Square Error: 7.813111558998616


In [ ]:
# DF2 smoking
# create the df to be trained.
df = df2[['lifespan','smoking']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.061034016735249
Interpret Root Mean Square Error: 7.598745651039806


In [ ]:
# DF2 alcohol
# create the df to be trained.
df = df2[['lifespan','alcohol']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.003543405583155801
Interpret Root Mean Square Error: 7.824074972232776


In [ ]:
# DF2 sugar
# create the df to be trained.
df = df2[['lifespan','sugar']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: -0.001953373219452992
Interpret Root Mean Square Error: 7.839463569279191


In [ ]:
# DF2 BMI
# create the df to be trained.
df = df2[['lifespan','BMI']].copy()

coefficient_R(df)

coefficient of determination(R²) vanilla: 0.005117058920511064
Interpret Root Mean Square Error: 7.818386016437573


In [ ]:
# df1

# create the df to be trained.
df = df2.drop(['length','sugar'], axis=1).copy()
 
coefficient_R(df)

coefficient of determination(R²) vanilla: 0.9799239512074865
Interpret Root Mean Square Error: 1.1138622504021247
